In [1]:
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import logging
import copy
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import kneighbors_graph

from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn import Parameter

from utils import utils
from utils.readers import InHospitalMortalityReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils


In [2]:
data_path = './data/'
small_part = False
arg_timestep = 1.0


In [3]:
data_path = './data/Challenge/normalized/'

train_x = pickle.load(open(data_path + 'train_x.dat', 'rb'))
train_y = pickle.load(open(data_path + 'train_y.dat', 'rb'))
train_x_len = pickle.load(open(data_path + 'train_x_len.dat', 'rb'))


dev_x = pickle.load(open(data_path + 'dev_x.dat', 'rb'))
dev_y = pickle.load(open(data_path + 'dev_y.dat', 'rb'))
dev_x_len = pickle.load(open(data_path + 'dev_x_len.dat', 'rb'))

test_x = pickle.load(open(data_path + 'test_x.dat', 'rb'))
test_y = pickle.load(open(data_path + 'test_y.dat', 'rb'))
test_x_len = pickle.load(open(data_path + 'test_x_len.dat', 'rb'))


print(len(train_x))
print(len(dev_x))
print(len(test_x))
print(sum(train_y)/len(train_y))
print(sum(dev_y)/len(dev_y))
print(sum(test_y)/len(test_y))

32268
4033
4035
0.07267261683401513
0.07265063228365981
0.07286245353159851


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
# device = torch.device('cuda')
print("available device: {}".format(device))

available device: cuda:0


In [5]:
def get_loss(y_pred, y_true):
    loss = torch.nn.BCELoss()
    return loss(y_pred, y_true)

In [6]:
def get_re_loss(y_pred, y_true):
    loss = torch.nn.MSELoss()
    return loss(y_pred, y_true)

In [7]:
def pad_sents(sents, pad_token):

    sents_padded = []

    max_length = max([len(_) for _ in sents])
    for i in sents:
        padded = list(i) + [pad_token]*(max_length-len(i))
        sents_padded.append(np.array(padded))


    return np.array(sents_padded)

In [8]:
def batch_iter(x, y, lens, batch_size, shuffle=False):
    """ Yield batches of source and target sentences reverse sorted by length (largest to smallest).
    @param data (list of (src_sent, tgt_sent)): list of tuples containing source and target sentence
    @param batch_size (int): batch size
    @param shuffle (boolean): whether to randomly shuffle the dataset
    """
    batch_num = math.ceil(len(x) / batch_size) 
    index_array = list(range(len(x)))

    if shuffle:
        np.random.shuffle(index_array)

    for i in range(batch_num):
        indices = index_array[i * batch_size: (i + 1) * batch_size] #  fetch out all the induces
        
        examples = []
        for idx in indices:
            examples.append((x[idx], y[idx],  lens[idx]))
       
        examples = sorted(examples, key=lambda e: len(e[0]), reverse=True)
    
        batch_x = [e[0] for e in examples]
        batch_y = [e[1] for e in examples]
#         batch_name = [e[2] for e in examples]
        batch_lens = [e[2] for e in examples]
       

        yield batch_x, batch_y, batch_lens

In [9]:
def length_to_mask(length, max_len=None, dtype=None):
    """length: B.
    return B x max_len.
    If max_len is None, then max of length will be used.
    """
    assert len(length.shape) == 1, 'Length shape should be 1 dimensional.'
    max_len = max_len or length.max().item()
    mask = torch.arange(max_len, device=length.device,
                        dtype=length.dtype).expand(len(length), max_len) < length.unsqueeze(1)
    if dtype is not None:
        mask = torch.as_tensor(mask, dtype=dtype, device=length.device)
    return mask

In [10]:
# K-NN cluster
def random_init(dataset, num_centers):
    num_points = dataset.size(0)
    dimension = dataset.size(1)

    indices = torch.tensor(np.array(random.sample(range(num_points), k=num_centers)), dtype=torch.long)

    indices = indices.to(device)
    centers = torch.gather(dataset, 0, indices.view(-1, 1).expand(-1, dimension))
    return centers

# Compute for each data point the closest center
def compute_codes(dataset, centers):
    num_points = dataset.size(0)
    dimension = dataset.size(1)
    num_centers = centers.size(0)
    # 5e8 should vary depending on the free memory on the GPU
    # Ideally, automatically ;)
    chunk_size = int(5e8 / num_centers)
    codes = torch.zeros(num_points, dtype=torch.long, device=device)
    centers_t = torch.transpose(centers, 0, 1)
    centers_norms = torch.sum(centers ** 2, dim=1).view(1, -1)
    for i in range(0, num_points, chunk_size):
        begin = i
        end = min(begin + chunk_size, num_points)
        dataset_piece = dataset[begin:end, :]
        dataset_norms = torch.sum(dataset_piece ** 2, dim=1).view(-1, 1)
        distances = torch.mm(dataset_piece, centers_t)
        distances *= -2.0
        distances += dataset_norms
        distances += centers_norms
        _, min_ind = torch.min(distances, dim=1)
        codes[begin:end] = min_ind
    return codes

# Compute new centers as means of the data points forming the clusters
def update_centers(dataset, codes, num_centers):
    num_points = dataset.size(0)
    dimension = dataset.size(1)
    centers = torch.zeros(num_centers, dimension, dtype=torch.float, device=device)
    cnt = torch.zeros(num_centers, dtype=torch.float, device=device)
    centers.scatter_add_(0, codes.view(-1, 1).expand(-1, dimension), dataset)
    cnt.scatter_add_(0, codes, torch.ones(num_points, dtype=torch.float, device=device))
    # Avoiding division by zero
    # Not necessary if there are no duplicates among the data points
    cnt = torch.where(cnt > 0.5, cnt, torch.ones(num_centers, dtype=torch.float, device=device))
    centers /= cnt.view(-1, 1)
    return centers

def cluster(dataset, num_centers):
    centers = random_init(dataset, num_centers)
    codes = compute_codes(dataset, centers)
    num_iterations = 0
    while True:
#         sys.stdout.write('.')
#         sys.stdout.flush()
#         num_iterations += 1
        centers = update_centers(dataset, codes, num_centers)
        new_codes = compute_codes(dataset, centers)
        # Waiting until the clustering stops updating altogether
        # This is too strict in practice
        if torch.equal(codes, new_codes):
#             sys.stdout.write('\n')
#             print('Converged in %d iterations' % num_iterations)
            break
        if num_iterations>1000:
            break
        codes = new_codes
    return centers, codes

In [11]:
# the concare model as our backbone: https://www.aaai.org/ojs/index.php/AAAI/article/download/5428/5284
class SingleAttention(nn.Module):
    def __init__(self, attention_input_dim, attention_hidden_dim, attention_type='add', demographic_dim=12, time_aware=False, use_demographic=False):
        super(SingleAttention, self).__init__()
        
        self.attention_type = attention_type
        self.attention_hidden_dim = attention_hidden_dim
        self.attention_input_dim = attention_input_dim
        self.use_demographic = use_demographic
        self.demographic_dim = demographic_dim
        self.time_aware = time_aware

        
        if attention_type == 'add':
            if self.time_aware == True:
                # self.Wx = nn.Parameter(torch.randn(attention_input_dim+1, attention_hidden_dim))
                self.Wx = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
                self.Wtime_aware = nn.Parameter(torch.randn(1, attention_hidden_dim))
                nn.init.kaiming_uniform_(self.Wtime_aware, a=math.sqrt(5))
            else:
                self.Wx = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
            self.Wt = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
            self.Wd = nn.Parameter(torch.randn(demographic_dim, attention_hidden_dim))
            self.bh = nn.Parameter(torch.zeros(attention_hidden_dim,))
            self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wd, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wx, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wt, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        elif attention_type == 'mul':
            self.Wa = nn.Parameter(torch.randn(attention_input_dim, attention_input_dim))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        elif attention_type == 'concat':
            if self.time_aware == True:
                self.Wh = nn.Parameter(torch.randn(2*attention_input_dim+1, attention_hidden_dim))
            else:
                self.Wh = nn.Parameter(torch.randn(2*attention_input_dim, attention_hidden_dim))

            self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wh, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        else:
            raise RuntimeError('Wrong attention type.')
        
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self, input, demo=None):
 
        batch_size, time_step, input_dim = input.size() # batch_size * time_step * hidden_dim(i)
       
        time_decays = torch.tensor(range(time_step-1,-1,-1), dtype=torch.float32).unsqueeze(-1).unsqueeze(0).to(device)# 1*t*1
        b_time_decays = time_decays.repeat(batch_size,1,1)# b t 1
        
        if self.attention_type == 'add': #B*T*I  @ H*I
            q = torch.matmul(input[:,-1,:], self.Wt)# b h
            q = torch.reshape(q, (batch_size, 1, self.attention_hidden_dim)) #B*1*H
            if self.time_aware == True:
                # k_input = torch.cat((input, time), dim=-1)
                k = torch.matmul(input, self.Wx)#b t h
                # k = torch.reshape(k, (batch_size, 1, time_step, self.attention_hidden_dim)) #B*1*T*H
                time_hidden = torch.matmul(b_time_decays, self.Wtime_aware)#  b t h
            else:
                k = torch.matmul(input, self.Wx)# b t h
                # k = torch.reshape(k, (batch_size, 1, time_step, self.attention_hidden_dim)) #B*1*T*H
            if self.use_demographic == True:
                d = torch.matmul(demo, self.Wd) #B*H
                d = torch.reshape(d, (batch_size, 1, self.attention_hidden_dim)) # b 1 h
            h = q + k + self.bh # b t h
            if self.time_aware == True:
                h += time_hidden
            h = self.tanh(h) #B*T*H
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step))# b t
        elif self.attention_type == 'mul':
            e = torch.matmul(input[:,-1,:], self.Wa)#b i
            e = torch.matmul(e.unsqueeze(1), input.permute(0,2,1)).squeeze() + self.ba #b t
        elif self.attention_type == 'concat':
            q = input[:,-1,:].unsqueeze(1).repeat(1,time_step,1)# b t i
            k = input
            c = torch.cat((q, k), dim=-1) #B*T*2I
            if self.time_aware == True:
                c = torch.cat((c, b_time_decays), dim=-1) #B*T*2I+1
            h = torch.matmul(c, self.Wh)
            h = self.tanh(h)
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step)) # b t 
        
   
        a = self.softmax(e) #B*T
        v = torch.matmul(a.unsqueeze(1), input).squeeze() #B*I

        return v, a

class FinalAttentionQKV(nn.Module):
    def __init__(self, attention_input_dim, attention_hidden_dim, attention_type='add', dropout=None):
        super(FinalAttentionQKV, self).__init__()
        
        self.attention_type = attention_type
        self.attention_hidden_dim = attention_hidden_dim
        self.attention_input_dim = attention_input_dim


        self.W_q = nn.Linear(attention_input_dim, attention_hidden_dim)
        self.W_k = nn.Linear(attention_input_dim, attention_hidden_dim)
        self.W_v = nn.Linear(attention_input_dim, attention_hidden_dim)

        self.W_out = nn.Linear(attention_hidden_dim, 1)

        self.b_in = nn.Parameter(torch.zeros(1,))
        self.b_out = nn.Parameter(torch.zeros(1,))

        nn.init.kaiming_uniform_(self.W_q.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_k.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_v.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_out.weight, a=math.sqrt(5))

        self.Wh = nn.Parameter(torch.randn(2*attention_input_dim, attention_hidden_dim))
        self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
        self.ba = nn.Parameter(torch.zeros(1,))
        
        nn.init.kaiming_uniform_(self.Wh, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        
        self.dropout = nn.Dropout(p=dropout)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=-1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input):
 
        batch_size, time_step, input_dim = input.size() # batch_size * input_dim + 1 * hidden_dim(i)
        input_q = self.W_q(torch.mean(input,1)) # b h
        input_k = self.W_k(input)# b t h
        input_v = self.W_v(input)# b t h

        if self.attention_type == 'add': #B*T*I  @ H*I

            q = torch.reshape(input_q, (batch_size, 1, self.attention_hidden_dim)) #B*1*H
            h = q + input_k + self.b_in # b t h
            h = self.tanh(h) #B*T*H
            e = self.W_out(h) # b t 1
            e = torch.reshape(e, (batch_size, time_step))# b t

        elif self.attention_type == 'mul':
            q = torch.reshape(input_q, (batch_size, self.attention_hidden_dim, 1)) #B*h 1
            e = torch.matmul(input_k, q).squeeze()#b t
            
        elif self.attention_type == 'concat':
            q = input_q.unsqueeze(1).repeat(1,time_step,1)# b t h
            k = input_k
            c = torch.cat((q, k), dim=-1) #B*T*2I
            h = torch.matmul(c, self.Wh)
            h = self.tanh(h)
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step)) # b t 
        
        a = self.softmax(e) #B*T
        if self.dropout is not None:
            a = self.dropout(a)
        v = torch.matmul(a.unsqueeze(1), input_v).squeeze() #B*I

        return v, a

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

def tile(a, dim, n_tile):
    init_dim = a.size(dim)
    repeat_idx = [1] * a.dim()
    repeat_idx[dim] = n_tile
    a = a.repeat(*(repeat_idx))
    order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)])).to(device)
    return torch.index_select(a, dim, order_index).to(device)

class PositionwiseFeedForward(nn.Module): # new added
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x)))), None



class PositionalEncoding(nn.Module): # new added / not use anymore
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=400):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0 
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)# b h t d_k
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k) # b h t t
    if mask is not None:# 1 1 t t
        scores = scores.masked_fill(mask == 0, -1e9)# b h t t 
    p_attn = F.softmax(scores, dim = -1)# b h t t
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn # b h t v (d_k) 
    
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, self.d_k * self.h), 3)
        self.final_linear = nn.Linear(d_model, d_model)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1) # 1 1 t t

        nbatches = query.size(0)# b
        input_dim = query.size(1)# i+1
        feature_dim = query.size(-1)# i+1

        #input size -> # batch_size * d_input * hidden_dim
        
        # d_model => h * d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))] # b num_head d_input d_k
        
       
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)# b num_head d_input d_v (d_k) 



      
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)# batch_size * d_input * hidden_dim



        return self.final_linear(x), torch.zeros(1).to(device)

class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-7):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

def cov(m, y=None):
    if y is not None:
        m = torch.cat((m, y), dim=0)
    m_exp = torch.mean(m, dim=1)
    x = m - m_exp[:, None]
    cov = 1 / (x.size(1) - 1) * x.mm(x.t())
    return cov

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        returned_value = sublayer(self.norm(x))
        return x + self.dropout(returned_value[0]) , returned_value[1]

class vanilla_transformer_encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.5):
        super(vanilla_transformer_encoder, self).__init__()

        # hyperparameters
        self.input_dim = input_dim  
        self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
#         self.PositionalEncoding = PositionalEncoding(self.d_model, dropout = 0, max_len = 400)

        self.GRUs = clones(nn.GRU(1, self.hidden_dim, batch_first = True), self.input_dim)
#         self.LastStepAttentions = clones(SingleAttention(self.hidden_dim, 8, attention_type='concat', demographic_dim=12, time_aware=True, use_demographic=False),self.input_dim)
        
        self.FinalAttentionQKV = FinalAttentionQKV(self.hidden_dim, self.hidden_dim, attention_type='mul',dropout = 1 - self.keep_prob)

        self.MultiHeadedAttention = MultiHeadedAttention(self.MHD_num_head, self.d_model,dropout = 1 - self.keep_prob)
        self.SublayerConnection = SublayerConnection(self.d_model, dropout = 1 - self.keep_prob)

        self.PositionwiseFeedForward = PositionwiseFeedForward(self.d_model, self.d_ff, dropout=0.1)

        
        self.to_query = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.to_key = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.to_value = nn.Linear(self.hidden_dim, self.hidden_dim)
        
        self.demo_proj_main = nn.Linear(12, self.hidden_dim)
        self.demo_proj = nn.Linear(12, self.hidden_dim)
        self.output = nn.Linear(self.hidden_dim, self.output_dim)

        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax(dim=-1)
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()
        
        self.bn = nn.BatchNorm1d(self.hidden_dim)

    def forward(self, input, lens):

        
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        assert(feature_dim == self.input_dim)# input Tensor : 256 * 48 * 76
        assert(self.d_model % self.MHD_num_head == 0)

       
        GRU_embeded_input = self.GRUs[0](pack_padded_sequence(input[:,:,0].unsqueeze(-1), lens, batch_first=True))[1].squeeze(0).unsqueeze(1) # b 1 h

        for i in range(feature_dim-1):
            embeded_input = self.GRUs[i+1](pack_padded_sequence(input[:,:,i+1].unsqueeze(-1), lens, batch_first=True))[1].squeeze(0).unsqueeze(1) # b 1 h
            GRU_embeded_input = torch.cat((GRU_embeded_input, embeded_input), 1)

        posi_input = self.dropout(GRU_embeded_input) # batch_size * d_input * hidden_dim

        contexts = self.SublayerConnection(posi_input, lambda x: self.MultiHeadedAttention(self.tanh(self.to_query(posi_input)), self.tanh(self.to_key(posi_input)), self.tanh(self.to_value(posi_input)), None))# # batch_size * d_input * hidden_dim
    
        DeCov_loss = contexts[1]
        contexts = contexts[0]

        contexts = self.SublayerConnection(contexts, lambda x: self.PositionwiseFeedForward(contexts))[0]# # batch_size * d_input * hidden_dim
      

        weighted_contexts = self.FinalAttentionQKV(contexts)[0]
        output = self.output(self.dropout(self.bn(weighted_contexts)))# b 1
        output = self.sigmoid(output)
          
        return output, DeCov_loss  ,weighted_contexts




In [12]:
# Our MAPLE framework
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(in_features, out_features).float())
        if bias:
            self.bias = Parameter(torch.Tensor(out_features).float())
        else:
            self.register_parameter('bias', None)
        self.initialize_parameters()

    def initialize_parameters(self):
        std = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-std, std)
        if self.bias is not None:
            self.bias.data.uniform_(-std, std)

    def forward(self, adj, x):
        y = torch.mm(x.float(), self.weight.float())
        output = torch.mm(adj.float(), y.float())
        if self.bias is not None:
            return output + self.bias.float()
        else:
            return output
        

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])
    

    

class MAPLE(nn.Module):
    def __init__(self, device, input_dim = 76, hidden_dim = 32, output_dim = 1, cluster_num = 12, dropout=0.0, block='LSTM'):
        super(MAPLE, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.cluster_num = cluster_num
        self.dropout = dropout
        self.block = block

        
        self.embed_layer = nn.Linear(self.input_dim, self.hidden_dim)
        self.GRUs = clones(nn.GRU(1, self.hidden_dim, batch_first = True), self.input_dim)
        self.opt_layer2 = nn.Linear( self.input_dim*self.hidden_dim, self.hidden_dim)
        
        self.opt_layer = nn.Linear(self.hidden_dim, self.output_dim)
        
        
        nn.init.xavier_uniform_(self.embed_layer.weight)
        nn.init.xavier_uniform_(self.opt_layer.weight)
        
        self.backbone = vanilla_transformer_encoder(input_dim = self.input_dim, hidden_dim = self.hidden_dim, d_model = self.hidden_dim,  MHD_num_head = 4 , d_ff = 2*self.hidden_dim, output_dim = self.output_dim).to(device)

        
        
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.bn1 = nn.BatchNorm1d(2 * self.hidden_dim)
        self.dropout = nn.Dropout(dropout) 
        self.weight1 = nn.Linear(self.hidden_dim, 1)
        self.weight2 = nn.Linear(self.hidden_dim, 1)
        self.GCN = GraphConvolution(self.hidden_dim, self.hidden_dim, bias=True)
        self.GCN.initialize_parameters()
        self.GCN_2 = GraphConvolution(self.hidden_dim, self.hidden_dim, bias=True)
        self.GCN_2.initialize_parameters()
        
        self.bn = nn.BatchNorm1d(self.hidden_dim)

        
    def sample_gumbel(self, shape, eps=1e-20):
        U = torch.rand(shape).to(device)

        return -torch.log(-torch.log(U + eps) + eps)


    def gumbel_softmax_sample(self, logits, temperature):
        y = logits + self.sample_gumbel(logits.size())
        return F.softmax(y / temperature, dim=-1)


    def gumbel_softmax(self, logits, temperature, hard=False):
        """
        ST-gumple-softmax
        input: [*, n_class]
        return: flatten --> [*, n_class] an one-hot vector
        """
        y = self.gumbel_softmax_sample(logits, temperature)

        if not hard:
            return y.view(-1, self.cluster_num)

        shape = y.size()
        _, ind = y.max(dim=-1)
        y_hard = torch.zeros_like(y).view(-1, shape[-1])
        y_hard.scatter_(1, ind.view(-1, 1), 1)
        y_hard = y_hard.view(*shape)
        # Set gradients w.r.t. y_hard gradients w.r.t. y
        y_hard = (y_hard - y).detach() + y
        return y_hard
    

        
    def forward(self, input, epoch, lens):
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        
 
        _,_, hidden_t = self.backbone(input,lens)
        
        centers, codes = cluster(hidden_t, self.cluster_num)
        
        if epoch == 0:
            A_mat = np.eye(self.cluster_num)
        else:
            A_mat = kneighbors_graph(np.array(centers.detach().cpu().numpy()), 20, mode='connectivity', include_self=False).toarray()
        
        adj_mat = torch.tensor(A_mat).to(device)
        
        e = self.relu(torch.matmul(hidden_t, centers.transpose(0, 1)) )# b clu_num

        scores = self.gumbel_softmax(e, temperature = 1, hard = True)
        digits = torch.argmax(scores, dim = -1)#  b


        h_prime = self.relu(self.GCN(adj_mat, centers))
        h_prime = self.relu(self.GCN_2(adj_mat, h_prime))


        clu_appendix = torch.matmul(scores, h_prime)
         
        weight1=torch.sigmoid(self.weight1(clu_appendix))
        weight2 = torch.sigmoid(self.weight2(hidden_t ))
        weight1 = weight1/(weight1+weight2)
        weight2= 1-weight1

        final_h = weight1*clu_appendix+weight2*hidden_t



        opt = self.sigmoid(self.opt_layer(self.bn(final_h)))
        
                      
        return opt, hidden_t

In [13]:
RANDOM_SEED = 2000
np.random.seed(RANDOM_SEED) #numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED) # cpu
torch.cuda.manual_seed(RANDOM_SEED) #gpu
torch.backends.cudnn.deterministic=True # cudnn

input_dim = 33
hidden_dim = 32
output_dim = 1
cluster_num = 100
dropout = 0.5
block = 'GRU'


model = MAPLE(device = device, input_dim = input_dim, hidden_dim = hidden_dim, output_dim = output_dim, cluster_num = cluster_num, dropout=dropout, block=block).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [14]:
file_name = './model/MAPLE-100-20-model'

batch_size = 256
epochs = 50
pad_token = np.zeros(33)
max_roc = 0
max_prc = 0
train_loss = []
train_model_loss = []
# train_decov_loss = []
valid_loss = []
valid_model_loss = []
# valid_decov_loss = []
history = []
np.set_printoptions(threshold=np.inf)
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)


for each_epoch in range(epochs):
    counter_batch = 0
    batch_loss = []
    model_batch_loss = []

    epoch_loss = []
    counter_batch = 0
    model.train() 
    
    for step, (batch_x, batch_y, batch_lens) in enumerate(batch_iter(train_x, train_y, train_x_len, batch_size, shuffle=True)):  

        counter_batch += len(batch_x)
        optimizer.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        if batch_x.shape[0] < cluster_num:
            continue
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()

        masks = length_to_mask(batch_lens).unsqueeze(-1).float()

        opt, _ = model(batch_x, each_epoch, batch_lens)
        
        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))


        model_loss =  BCE_Loss 

        loss = model_loss
        
        batch_loss.append(loss.cpu().detach().numpy())
        model_batch_loss.append(model_loss.cpu().detach().numpy())

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 20)
        optimizer.step()
        
        if step % 30 == 0:
            print('Epoch %d Batch %d: Train Loss = %.4f'%(each_epoch, step, np.mean(np.array(batch_loss))))
    train_loss.append(np.mean(np.array(batch_loss)))
    train_model_loss.append(np.mean(np.array(model_batch_loss)))

    

    batch_loss = []
    model_batch_loss = []
    
    
    y_true = []
    y_pred = []
    with torch.no_grad():
        model.eval()
        for batch_x, batch_y, batch_lens in batch_iter(dev_x, dev_y, dev_x_len, 256):
            batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
            if batch_x.shape[0] < cluster_num:
                continue
            batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
            batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
            masks = length_to_mask(batch_lens).unsqueeze(-1).float()
     
            opt, _ = model(batch_x, 100, batch_lens)
        
            model_loss = get_loss(opt, batch_y.unsqueeze(-1)) 


            loss = model_loss
            
            batch_loss.append(loss.cpu().detach().numpy())
            model_batch_loss.append(model_loss.cpu().detach().numpy())
            y_pred += list(opt.cpu().detach().numpy().flatten())
            y_true += list(batch_y.cpu().numpy().flatten())
            
    valid_loss.append(np.mean(np.array(batch_loss)))
    valid_model_loss.append(np.mean(np.array(model_batch_loss)))
    
    print("\n==>Predicting on validation")
    print('Valid Loss = %.4f'%(valid_loss[-1]))
    print('valid_model Loss = %.4f'%(valid_model_loss[-1]))

    y_pred = np.array(y_pred)
    y_pred = np.stack([1 - y_pred, y_pred], axis=1)
    ret = metrics.print_metrics_binary(y_true, y_pred)
    history.append(ret)
    print('')

    
    cur_auroc = ret['auroc']
    cur_prc = ret['auprc']
    
    if cur_prc > max_prc:
        max_prc = cur_prc
        state = {
            'net': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': each_epoch
        }
        torch.save(state, file_name+"prc")
        print('\n------------ Save best-prc model ------------\n')
        
print('==============DONE==============')

Epoch 0 Batch 0: Train Loss = 0.7079
Epoch 0 Batch 30: Train Loss = 0.6272
Epoch 0 Batch 60: Train Loss = 0.5758
Epoch 0 Batch 90: Train Loss = 0.5293
Epoch 0 Batch 120: Train Loss = 0.4884

==>Predicting on validation
Valid Loss = 6.1226
valid_model Loss = 6.1226
confusion matrix:
[[3740    0]
 [ 293    0]]
accuracy = 0.9273493885993958
precision class 0 = 0.9273493885993958
precision class 1 = nan
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.5432069135441953
AUC of PRC = 0.0913012870160394
min(+P, Se) = 0.1346153846153846
f1_score = nan


------------ Save best-prc model ------------



/home/other/choczhang/EMR/challenge/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])


Epoch 1 Batch 0: Train Loss = 0.3684
Epoch 1 Batch 30: Train Loss = 0.3072
Epoch 1 Batch 60: Train Loss = 0.2882
Epoch 1 Batch 90: Train Loss = 0.2753
Epoch 1 Batch 120: Train Loss = 0.2692

==>Predicting on validation
Valid Loss = 0.2707
valid_model Loss = 0.2707
confusion matrix:
[[3583  157]
 [ 230   63]]
accuracy = 0.9040416479110718
precision class 0 = 0.9396800398826599
precision class 1 = 0.2863636314868927
recall class 0 = 0.9580214023590088
recall class 1 = 0.21501706540584564
AUC of ROC = 0.7805378620576371
AUC of PRC = 0.22781390046352282
min(+P, Se) = 0.2721311475409836
f1_score = 0.2456140409586306


------------ Save best-prc model ------------

Epoch 2 Batch 0: Train Loss = 0.1857
Epoch 2 Batch 30: Train Loss = 0.2282
Epoch 2 Batch 60: Train Loss = 0.2260
Epoch 2 Batch 90: Train Loss = 0.2226
Epoch 2 Batch 120: Train Loss = 0.2175

==>Predicting on validation
Valid Loss = 0.2203
valid_model Loss = 0.2203
confusion matrix:
[[3631  109]
 [ 207   86]]
accuracy = 0.921646416


==>Predicting on validation
Valid Loss = 0.1331
valid_model Loss = 0.1331
confusion matrix:
[[3714   26]
 [ 145  148]]
accuracy = 0.9575998187065125
precision class 0 = 0.9624254703521729
precision class 1 = 0.8505747318267822
recall class 0 = 0.9930481314659119
recall class 1 = 0.5051194429397583
AUC of ROC = 0.9383712653537992
AUC of PRC = 0.7364275044433395
min(+P, Se) = 0.6791808873720137
f1_score = 0.6338329731231097


------------ Save best-prc model ------------

Epoch 14 Batch 0: Train Loss = 0.1003
Epoch 14 Batch 30: Train Loss = 0.1206
Epoch 14 Batch 60: Train Loss = 0.1249
Epoch 14 Batch 90: Train Loss = 0.1308
Epoch 14 Batch 120: Train Loss = 0.1307

==>Predicting on validation
Valid Loss = 0.1326
valid_model Loss = 0.1326
confusion matrix:
[[3708   32]
 [ 139  154]]
accuracy = 0.9575998187065125
precision class 0 = 0.9638679623603821
precision class 1 = 0.8279569745063782
recall class 0 = 0.9914438724517822
recall class 1 = 0.5255972743034363
AUC of ROC = 0.93429121571060

Epoch 26 Batch 60: Train Loss = 0.1168
Epoch 26 Batch 90: Train Loss = 0.1201
Epoch 26 Batch 120: Train Loss = 0.1208

==>Predicting on validation
Valid Loss = 0.1230
valid_model Loss = 0.1230
confusion matrix:
[[3707   33]
 [ 133  160]]
accuracy = 0.9588395953178406
precision class 0 = 0.9653645753860474
precision class 1 = 0.8290155529975891
recall class 0 = 0.9911764860153198
recall class 1 = 0.5460751056671143
AUC of ROC = 0.9411646073260207
AUC of PRC = 0.7464789368283604
min(+P, Se) = 0.673469387755102
f1_score = 0.6584362601450193

Epoch 27 Batch 0: Train Loss = 0.1120
Epoch 27 Batch 30: Train Loss = 0.1249
Epoch 27 Batch 60: Train Loss = 0.1124
Epoch 27 Batch 90: Train Loss = 0.1168
Epoch 27 Batch 120: Train Loss = 0.1182

==>Predicting on validation
Valid Loss = 0.1252
valid_model Loss = 0.1252
confusion matrix:
[[3722   18]
 [ 141  152]]
accuracy = 0.9605752825737
precision class 0 = 0.9634998440742493
precision class 1 = 0.8941176533699036
recall class 0 = 0.9951871633529663

Epoch 39 Batch 60: Train Loss = 0.1104
Epoch 39 Batch 90: Train Loss = 0.1103
Epoch 39 Batch 120: Train Loss = 0.1113

==>Predicting on validation
Valid Loss = 0.1136
valid_model Loss = 0.1136
confusion matrix:
[[3714   26]
 [ 119  174]]
accuracy = 0.9640465974807739
precision class 0 = 0.9689538478851318
precision class 1 = 0.8700000047683716
recall class 0 = 0.9930481314659119
recall class 1 = 0.5938566327095032
AUC of ROC = 0.9501487470569985
AUC of PRC = 0.7796814264063661
min(+P, Se) = 0.7220338983050848
f1_score = 0.7058823098172051


------------ Save best-prc model ------------

Epoch 40 Batch 0: Train Loss = 0.1683
Epoch 40 Batch 30: Train Loss = 0.1114
Epoch 40 Batch 60: Train Loss = 0.1082
Epoch 40 Batch 90: Train Loss = 0.1120
Epoch 40 Batch 120: Train Loss = 0.1127

==>Predicting on validation
Valid Loss = 0.1172
valid_model Loss = 0.1172
confusion matrix:
[[3687   53]
 [ 101  192]]
accuracy = 0.9618149995803833
precision class 0 = 0.9733368754386902
precision class 1 = 0.

In [15]:
checkpoint = torch.load(file_name)
save_epoch = checkpoint['epoch']
print("last saved model is in epoch {}".format(save_epoch))
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

In [16]:
batch_loss = []
y_true = []
y_pred = []
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_lens) in enumerate(batch_iter(test_x, test_y, test_x_len, batch_size, shuffle=False)):  
        optimizer.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
#         batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)

        masks = length_to_mask(batch_lens).unsqueeze(-1).float()

        opt,_ = model(batch_x, 100, batch_lens)

        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)

        model_loss =  BCE_Loss 

        loss = model_loss
        batch_loss.append(loss.cpu().detach().numpy())
        if step % 20 == 0:
            print('Batch %d: Test Loss = %.4f'%(step, loss.cpu().detach().numpy()))
        y_pred += list(opt.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\n==>Predicting on test")
print('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

last saved model is in epoch 32
Batch 0: Test Loss = 0.1203

==>Predicting on test
Test Loss = 0.1066
confusion matrix:
[[3698   43]
 [ 103  191]]
accuracy = 0.9638165831565857
precision class 0 = 0.9729018807411194
precision class 1 = 0.8162392973899841
recall class 0 = 0.9885057210922241
recall class 1 = 0.6496598720550537
AUC of ROC = 0.9567651706499225
AUC of PRC = 0.8044356173079956
min(+P, Se) = 0.7210884353741497
f1_score = 0.7234848166916005


In [17]:
batch_loss = []
y_true = []
y_pred = []
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_lens) in enumerate(batch_iter(test_x, test_y, test_x_len, batch_size, shuffle=False)):  
        optimizer.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
#         batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)

        masks = length_to_mask(batch_lens).unsqueeze(-1).float()

        opt,_ = model(batch_x, 100, batch_lens)

        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)

        model_loss =  BCE_Loss 

        loss = model_loss
        batch_loss.append(loss.cpu().detach().numpy())
        if step % 20 == 0:
            print('Batch %d: Test Loss = %.4f'%(step, loss.cpu().detach().numpy()))
        y_pred += list(opt.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\n==>Predicting on test")
print('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

Batch 0: Test Loss = 0.1203

==>Predicting on test
Test Loss = 0.1066
confusion matrix:
[[3698   43]
 [ 103  191]]
accuracy = 0.9638165831565857
precision class 0 = 0.9729018807411194
precision class 1 = 0.8162392973899841
recall class 0 = 0.9885057210922241
recall class 1 = 0.6496598720550537
AUC of ROC = 0.9567651706499225
AUC of PRC = 0.8044356173079956
min(+P, Se) = 0.7210884353741497
f1_score = 0.7234848166916005


In [18]:
# Bootstrap
N = len(y_true)
N_idx = np.arange(N)
K = 1000

auroc = []
auprc = []
minpse = []
for i in range(K):
    boot_idx = np.random.choice(N_idx, N, replace=True)
    boot_true = np.array(y_true)[boot_idx]
    boot_pred = y_pred[boot_idx, :]
    test_ret = metrics.print_metrics_binary(boot_true, boot_pred, verbose=0)
    auroc.append(test_ret['auroc'])
    auprc.append(test_ret['auprc'])
    minpse.append(test_ret['minpse'])
#     print('%d/%d'%(i+1,K))
    
print('auroc %.4f(%.4f)'%(np.mean(auroc), np.std(auroc)))
print('auprc %.4f(%.4f)'%(np.mean(auprc), np.std(auprc)))
print('minpse %.4f(%.4f)'%(np.mean(minpse), np.std(minpse)))

auroc 0.9565(0.0064)
auprc 0.8043(0.0201)
minpse 0.7213(0.0214)
